In [1]:
pip install smolagents litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2


In [49]:
from smolagents import LiteLLMModel , CodeAgent

model = LiteLLMModel(
    "gemini/gemini-2.0-flash",
    temperature=0.2,
    api_key="",
    max_token = 8000
)

In [33]:
from smolagents import LiteLLMModel , CodeAgent

model = LiteLLMModel(
    "gemini/gemini-2.0-flash",
    temperature=0.2,
    api_key="",
    max_token = 8000
)
from smolagents import Tool
import requests
import json

class TodoManagerTool(Tool):
    name = "todo_manager"
    description = (
        "Manages todo items by interacting with the specified API. Supports creating, retrieving, "
        "updating, and deleting todo items. Returns responses as JSON strings."
    )

    inputs = {
        "operation": {
            "type": "string",
            "description": "The operation to perform: 'create', 'retrieve', 'update', or 'delete'."
        },
        "title": {
            "type": "string",
            "description": "The title of the todo item (required for 'create' and 'update').",
            "default": None,
            "nullable": True
        },
        "description": {
            "type": "string",
            "description": "The description of the todo item (required for 'create' and 'update').",
            "default": None,
            "nullable": True
        },
        "completed": {
            "type": "boolean",
            "description": "The completion status of the todo item (default: false for 'create' and 'update').",
            "default": False,
            "nullable": True
        },
        "id": {
            "type": "integer",
            "description": "The ID of the todo item (required for 'update' and 'delete').",
            "default": None,
            "nullable": True
        },
        "skip": {
            "type": "integer",
            "description": "Number of items to skip for pagination (default: 0 for 'retrieve').",
            "default": 0,
            "nullable": True
        },
        "limit": {
            "type": "integer",
            "description": "Maximum number of items to return (default: 100 for 'retrieve').",
            "default": 100,
            "nullable": True
        }
    }

    output_type = "string"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.base_url = "https://aiexecutiveassistantagent-production.up.railway.app/todos/"
        self.headers = {
            "accept": "application/json",
            "Content-Type": "application/json"
        }
        self.delete_headers = {
            "accept": "*/*"
        }

    def forward(self, operation: str, title: str = None, description: str = None,
                completed: bool = False, id: int = None, skip: int = 0, limit: int = 100) -> str:
        """
        Perform the specified todo operation.

        Args:
            operation (str): The operation to perform ('create', 'retrieve', 'update', 'delete').
            title (str, optional): The title of the todo item (for 'create', 'update').
            description (str, optional): The description of the todo item (for 'create', 'update').
            completed (bool, optional): The completion status (for 'create', 'update').
            id (int, optional): The ID of the todo item (for 'update', 'delete').
            skip (int, optional): Number of items to skip (for 'retrieve').
            limit (int, optional): Maximum number of items to return (for 'retrieve').

        Returns:
            str: A JSON string of the API response or an error message.
        """
        # Validate operation
        valid_operations = {"create", "retrieve", "update", "delete"}
        if operation not in valid_operations:
            return f"Error: Invalid operation. Must be one of {valid_operations}."

        try:
            if operation == "create":
                # Validate inputs for create
                if not title or not isinstance(title, str):
                    return "Error: Title must be a non-empty string for create operation."
                if not description or not isinstance(description, str):
                    return "Error: Description must be a non-empty string for create operation."
                if not isinstance(completed, bool):
                    return "Error: Completed must be a boolean for create operation."

                # Construct payload
                payload = {
                    "title": title.strip(),
                    "description": description.strip(),
                    "completed": completed
                }

                # Send POST request
                response = requests.post(self.base_url, headers=self.headers, json=payload)
                response.raise_for_status()
                return json.dumps(response.json(), indent=2)

            elif operation == "retrieve":
                # Validate inputs for retrieve
                if not isinstance(skip, int) or skip < 0:
                    return "Error: Skip must be a non-negative integer for retrieve operation."
                if not isinstance(limit, int) or limit < 1:
                    return "Error: Limit must be a positive integer for retrieve operation."

                # Construct query parameters
                querystring = {"skip": str(skip), "limit": str(limit)}

                # Send GET request
                response = requests.get(self.base_url, headers=self.headers, params=querystring)
                response.raise_for_status()
                return json.dumps(response.json(), indent=2)

            elif operation == "update":
                # Validate inputs for update
                if id is None or not isinstance(id, int) or id < 1:
                    return "Error: ID must be a positive integer for update operation."
                if not title or not isinstance(title, str):
                    return "Error: Title must be a non-empty string for update operation."
                if not description or not isinstance(description, str):
                    return "Error: Description must be a non-empty string for update operation."
                if not isinstance(completed, bool):
                    return "Error: Completed must be a boolean for update operation."

                # Construct payload
                payload = {
                    "title": title.strip(),
                    "description": description.strip(),
                    "completed": completed
                }

                # Construct URL with ID
                url = f"{self.base_url}{id}"

                # Send PUT request
                response = requests.put(url, headers=self.headers, json=payload)
                response.raise_for_status()
                return json.dumps(response.json(), indent=2)

            elif operation == "delete":
                # Validate inputs for delete
                if id is None or not isinstance(id, int) or id < 1:
                    return "Error: ID must be a positive integer for delete operation."

                # Construct URL with ID
                url = f"{self.base_url}{id}"

                # Send DELETE request
                response = requests.delete(url, headers=self.delete_headers)
                response.raise_for_status()
                # DELETE may return no content (204), so return a success message
                if response.status_code == 204:
                    return json.dumps({"message": f"Todo item with ID {id} deleted successfully"}, indent=2)
                return json.dumps(response.json(), indent=2)

        except requests.RequestException as e:
            return f"Error performing {operation} operation: {str(e)}"
        except json.JSONDecodeError:
            return "Error: Invalid JSON response from the API."
        except Exception as e:
            return f"Error processing request: {str(e)}"


In [39]:
create_todo = TodoManagerTool()
manager_agent = CodeAgent(
    tools=[create_todo],
    model=model,
    max_steps=10,
    additional_authorized_imports=['json','math', 'statistics', 'datetime', 'collections', 'queue', 'random', 're',
'unicodedata', 'itertools', 'time', 'stat']

)

In [48]:
manager_agent.run("Delete all todo except last 5.  if id not povided then call get find the appropriate id the do the main process")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Delete all todo except last 5.  if id not povided then call get find the appropriate id the do the main process │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import json                                                                                                      
                                                                                                                   
  all_todos = todo_manager(operation='retrieve', title=None, description=None, completed=None, id=None, skip=0,    
  limit=100)                                                                                                       
  print(all_todos)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[
  {
    "title": "World Cup - Match Day 1",
    "description": "Important match scheduled for World Cup on 2025-05-08",
    "completed": false,
    "id": 13,
    "created_at": "2025-05-08T08:47:46.427229Z",
    "updated_at": null
  },
  {
    "title": "World Cup - Match Day 2",
    "description": "Important match scheduled for World Cup on 2025-05-09",
    "completed": false,
    "id": 14,
    "created_at": "2025-05-08T08:47:49.436896Z",
    "updated_at": null
  },
  {
    "title": "World Cup - Match Day 3",
    "description": "Important match scheduled for World Cup on 2025-05-10",
    "completed": false,
    "id": 15,
    "created_at": "2025-05-08T08:47:52.467255Z",
    "updated_at": null
  },
  {
    "title": "World Cup - Match Day 4",
    "description": "Important match scheduled for World Cup on 2025-05-11",
    "completed": false,
    "id": 16,
    "created_at": "2025-05-08T08:47:55.727937Z",
    "updated_at": null
  },
  {
    "title": "World Cup - Match Day 5",
    "description": "Important match scheduled for World Cup on 2025-05-12",
    "completed": false,
    "id": 17,
    "created_at": "2025-05-08T08:47:58.744925Z",
    "updated_at": null
  },
  {
    "title": "T20 World Cup - Match Day 1",
    "description": "Important match scheduled for T20 World Cup on 2025-05-08",
    "completed": false,
    "id": 18,
    "created_at": "2025-05-08T08:48:02.022852Z",
    "updated_at": null
  },
  {
    "title": "T20 World Cup - Match Day 2",
    "description": "Important match scheduled for T20 World Cup on 2025-05-09",
    "completed": false,
    "id": 19,
    "created_at": "2025-05-08T08:48:05.024592Z",
    "updated_at": null
  },
  {
    "title": "T20 World Cup - Match Day 3",
    "description": "Important match scheduled for T20 World Cup on 2025-05-10",
    "completed": false,
    "id": 20,
    "created_at": "2025-05-08T08:48:08.272530Z",
    "updated_at": null
  },
  {
    "title": "T20 World Cup - Match Day 4",
    "description": "Important match scheduled for T20 World Cup on 2025-05-11",
    "completed": false,
    "id": 21,
    "created_at": "2025-05-08T08:48:11.297211Z",
    "updated_at": null
  },
  {
    "title": "T20 World Cup - Match Day 5",
    "description": "Important match scheduled for T20 World Cup on 2025-05-12",
    "completed": false,
    "id": 22,
    "created_at": "2025-05-08T08:48:14.542487Z",
    "updated_at": null
  },
  {
    "title": "Champions Trophy - Match Day 1",
    "description": "Important match scheduled for Champions Trophy on 2025-05-08",
    "completed": false,
    "id": 23,
    "created_at": "2025-05-08T08:48:17.568659Z",
    "updated_at": null
  },
  {
    "title": "Champions Trophy - Match Day 2",
    "description": "Important match scheduled for Champions Trophy on 2025-05-09",
    "completed": false,
    "id": 24,
    "created_at": "2025-05-08T08:48:20.839878Z",
    "updated_at": null
  },
  {
    "title": "Champions Trophy - Match Day 3",
    "description": "Important match scheduled for Champions Trophy on 2025-05-10",
    "completed": false,
    "id": 25,
    "created_at": "2025-05-08T08:48:23.848659Z",
    "updated_at": null
  },
  {
    "title": "Champions Trophy - Match Day 4",
    "description": "Important match scheduled for Champions Trophy on 2025-05-11",
    "completed": false,
    "id": 26,
    "created_at": "2025-05-08T08:48:26.863257Z",
    "updated_at": null
  },
  {
    "title": "Champions Trophy - Match Day 5",
    "description": "Important match scheduled for Champions Trophy on 2025-05-12",
    "completed": false,
    "id": 27,
    "created_at": "2025-05-08T08:48:29.868874Z",
    "updated_at": null
  },
  {
    "title": "Asia Cup - Match Day 1",
    "description": "Important match scheduled for Asia Cup on 2025-05-08",
    "completed": false,
    "id": 28,
    "created_at": "2025-05-08T08:48:33.116001Z",
    "updated_at": null
  },
  {
    "title": "Asia Cup - Match Day 2",
    "description": "Important match scheduled for Asia Cup on 2025-05-0

[Step 1: Duration 3.24 seconds| Input tokens: 2,400 | Output tokens: 102]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Todos deleted successfully, except the last 5.")                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Todos deleted successfully, except the last 5.

[Step 3: Duration 1.06 seconds| Input tokens: 15,340 | Output tokens: 314]

'Todos deleted successfully, except the last 5.'